In [1]:
!pip install 'biopandas'
!pip install scipy
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip


In [2]:
import argparse
import collections
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
import subprocess
import time
from biopandas.pdb import PandasPdb
from sklearn.neighbors import KDTree
from collections import defaultdict
import csv
import pandas as pd

In [3]:
!pwd

/home/rnomura/research/analysis/analysis/analysis_program


In [4]:
data_path = "/home/rnomura/research/analysis/analysis/pdb_files/"
itp_path = "/home/rnomura/research/analysis/analysis/itp_files/"
research_path = "/home/rnomura/research/analysis/analysis/analysis_program/csv/"
pic_path = "/home/rnomura/research/analysis/analysis/analysis_program/pic_stock/"

In [8]:
class dipole_calc:
  def __init__(self,keyword,flag):
    self.memo_charge=[]
    self.keyword = keyword
    self.flag = flag
    self.zahyo_r = []
  def charge(self):  
    memo_lines = []
    with open(itp_path + self.keyword + '.itp','r') as f:
      flag = 0
      for line in f:    
        if '[ atoms ]' in line:
          flag = 1
        elif '[ position_restraints ]' in line:
          break
        elif flag:
          memo_lines.append(line.rstrip('\n'))
    #memo_linesの余計なものを除去
    last_index = len(memo_lines) - 2
    flag = 0
    for index, line in enumerate(memo_lines):
        amino = line[12:15]
        residue_num = line[10:12]
        if (residue_num == "1" and (amino == "GLU" or amino == "ASP")):#N末端にGLUとASPがあるときは非電荷
            self.memo_charge.append(0)
        if (residue_num == "56" and (amino == "GLU" or amino == "ASP")):#C末端にLYSとARGがあるときは非電荷
            self.memo_charge.append(0)
        if line:
            self.memo_charge.append(float(line[-5:-1]))
    return self.memo_charge
  def zahyo_converter(self):
    pdb = PandasPdb().read_pdb(data_path+self.keyword+'_cg.pdb')
    zahyo = pdb.df['ATOM'][['x_coord','y_coord','z_coord']].values
    center_point = zahyo.mean(axis=0)
    self.zahyo_r = zahyo - center_point#座標中心を原点にし、座標変換
    return self.zahyo_r
  def calc(self):
    if self.flag == 0:
      vector = [0,0,0]
      for a,b in zip(self.zahyo_converter(),self.charge()):
        r = a
        q = b
        vector += r*q
      dipole = 4.803*vector
      scolar = (dipole[0]**2 + dipole[1]**2 + dipole[2]**2)
      scolar = scolar **0.5
      return scolar#dipoleの大きさ
    else:
      quadropole_x = 0
      quadropole_y = 0
      quadropole_z = 0
      for a,b in zip(self.zahyo_converter(),self.charge()):
        r= (a[0]**2+a[1]**2+a[2]**2)**1/2
        q = b
        quadropole_x += 0.5*q*(3*a[0]**2-r**2)
        quadropole_y += 0.5*q*(3*a[1]**2-r**2)
        quadropole_z += 0.5*q*(3*a[2]**2-r**2)
        quadropole_sum = quadropole_x + quadropole_y + quadropole_z
      if (quadropole_sum < 0):
        quadropole_sum *= -1
      return (quadropole_sum)**0.5#quadropoleの大きさ?
  def __del__(self):#デストラクターでメモリ解放
    pass

In [11]:
#print(dipole_calc("minus1",0).calc())

In [54]:
#dipole
for name in ["minus1","minus2","plus1","plus2","plus2_ala_0","plus2_ala_A","sample4","sample5", "sample6", "sample7", "sample8", "sample9"]:
    print(dipole_calc(name,0).calc())

113.46597151491669
112.97898483533938
173.95496976759526
135.0610931974097
0.0
237.48409623409984
27.828583657921953
307.50373447082757
264.7315289164652
129.42306520288554
231.03355722257567
212.28275584939578


In [10]:
#quadropole->ここでエラーが出ている理由が分からない。
for name in ["minus1","minus2","plus1","plus2","plus2_ala_0","plus2_ala_A","sample4","sample5", "sample6", "sample7", "sample8", "sample9"]:
    print(dipole_calc(name,1).calc())

152.964687107665
374.06971810004967
432.5019626126414
283.24033493623114
0.0
103.04979149367932
229.161950521503
239.9143291731847
304.1618752256726
283.54983581443287
164.14659994065923
268.5497403931765
